In [ ]:
import tabula
import textract
import re
import pymysql
from os import listdir
from os.path import isfile, join
import pandas as pd
from IPython.display import display

In [ ]:
countries = []
def get_country(country):
    if country not in countries:
        print('================')
        print('Country Inserted::: %s', country)
        countries.append(country)
#     curs = CONN.cursor(pymysql.cursors.DictCursor)
    
#     find_sql = '''select id from team where lower(country) like %s'''
#     curs.execute(find_sql, ('%'+country+'%'))
#     result = curs.fetchone()
#     return result

players = []
# def get_player_id(country_id, num):
# #     print(str(country_id)+';;;;'+str(num))
#     curs = CONN.cursor(pymysql.cursors.DictCursor)
    
#     find_sql = '''select * from player where team_id=%s and num=%s'''
#     curs.execute(find_sql, (country_id, num))
#     result = curs.fetchone()
#     return result    

player_stats = []
def save_stat(player_stat):
    player_stats.append(player_stat)
#     print(player_stat)
#     curs = CONN.cursor(pymysql.cursors.DictCursor)
#     insert_sql = """insert into player_stat (player_id, match_id, stat_type, stat_value, half) values (%s, %s, %s, %s, %s)"""
#     curs.execute(insert_sql, (player_stat['player_id'], player_stat['match_id'], player_stat['stat_type'], player_stat['stat_value'], player_stat['half'],))
#     CONN.commit()

In [ ]:
# root_dirs =['/Users/gracelee/football_worldcup', #group phase
#             '/Users/gracelee/football_worldcup_knock']
root_dirs = ['/Users/grace/workspace/football/russia_dataset/']

In [ ]:
country_names = ['england', 'brazil', 'argentina', 'switzerland', 'uruguay', 'portugal', 'croatia',
                 'sweden', 'brazil', 'panama', 'mexico', 'nigeria', 'egypt', 'poland', 
                 'iceland', 'belgium', 'japan', 'costa rica', 'france', 'serbia', 'spain',
                 'denmark', 'korea republic', 'germany', 'saudi arabia', 'tunisia', 'russia', 'colombia',
                 'australia', 'senegal', 'peru', 'morocco', 'ir iran', 'costa rica'
                ]

In [ ]:
for _dir in root_dirs:
    onlyfiles = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    onlyfiles = [_dir+'/'+x for x in onlyfiles if ('teamstatistics' in x and 'fwc' not in x)]
    onlyfiles.sort()

#     for z in range(len(onlyfiles)):
#         print('file z::: %s' % str(z))
#         print(onlyfiles[z])
#         player_statistic = textract.process(onlyfiles[z])
#         lines = str(player_statistic).split('\\n')
#         country_id = lines[9].lower().strip()
#         if country_id not in country_names:
#             country_id = lines[13].lower().strip()+'%'
# #         print(country_id)

#         if lines[7].find('#')>=0:
#             match_id = lines[7].split(' ')[1]
#         else:
#             match_id = lines[11].split(' ')[1]
#         print('%s :::: %s or %s' % (country_id, lines[9], lines[13]))

#         df = tabula.read_pdf(onlyfiles[z], multiple_tables=False)

In [ ]:
len(onlyfiles)

In [ ]:
import numpy as np

In [ ]:
dfs = {}

for i in range(len(onlyfiles[110:])):
    c_file = onlyfiles[i+110]
    print(c_file)
    player_statistic = textract.process(c_file)
    lines = str(player_statistic).split('\\n')
    country_id = lines[9].lower().strip()
    if country_id not in country_names:
        country_id = lines[13].lower().strip()+'%'
    #         print(country_id)

    if lines[7].find('#')>=0:
        match_id = lines[7].split(' ')[1]
    else:
        match_id = lines[11].split(' ')[1]
    print('%s :::: %s or %s' % (country_id, lines[9], lines[13]))

    df = tabula.read_pdf(c_file, multiple_tables=False)
    df = df.replace(np.nan, '', regex=True)
    
    pos_index = 16
    if df.tail(1).index[0]<pos_index:
        pos_index = df.tail(1).index[0]
    try:
        int(df.loc[pos_index][0])
        end_index = 16
    except ValueError:
        end_index = pos_index
        
    print('end_index' + str(end_index))
    df = df[2:end_index]
#     df.reset_index(inplace=True)
    print('columns : ' + str(len(df.columns)))
    initia_col_len = len(df.columns)
    if len(df.columns)==13:
        df.columns = ['No', 'Name', '-', 'Time Distance covered (metres)', 'Distance covered (metres)', \
                      'Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.',\
                     'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent', 'Activity time spent Zone 5']

        if len(df.iloc[0]['Time Distance covered (metres)'].split(' '))>1:
            df['Distance covered (metres) Total'] = df['Time Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
            df['Time'] = df['Time Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) In Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) Not in Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
        else:
            df['Time'] = df['Time Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) Total'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) In Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
            df['Distance covered (metres) Not in Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2])
        
    else:
        if len(df.columns)==10:
            df.columns = ['No', 'Name', 'Time', 'Distance covered (metres)', 'Time spent',\
                     'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent', 'Activity time spent Zone 5']
        elif len(df.columns)==11:
            df.columns = ['No', 'Name', 'Time', 'Distance covered (metres)', 'Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.',\
                     'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent']
        else:
            df.columns = ['No', 'Name', 'Time', 'Distance covered (metres)', 'Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.',\
                     'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent', 'Activity time spent Zone 5']
        
        for ind, row in df.iterrows():
            if row['Name'].find('...')>=0:
                print(ind)
                print(row['Name'])
                name_arr = row['Name'].split('...')
                print(name_arr)
                df.loc[ind, 'Time'] = str(name_arr[1])+' '+str(row['Time'])
                df.loc[ind, 'Name'] = str(name_arr[0])+'...'
        print(len(str(df.iloc[0]['Time']).split(' ')))
        
        if len(str(df.iloc[0]['Time']).split(' '))>1:
            df['Distance covered (metres) Total'] = df['Time'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
            df['Time'] = df['Time'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) In Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
            df['Distance covered (metres) Not in Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
        else:
            df['Distance covered (metres) Total'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=1 else '')
            df['Distance covered (metres) In Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1] if len(str(x).split(' '))>=2 else '')
            df['Distance covered (metres) Not in Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2] if len(str(x).split(' '))>=3 else '')

    
    if initia_col_len==11:
        print('under 11')
#         len(max(df['Activity time spent'].str.split(' '), key=len))
        df['Activity time spent Zone 2'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=1 else '')
        df['Activity time spent Zone 3'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1] if len(str(x).split(' '))>=2 else '')
        df['Activity time spent Zone 4'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2] if len(str(x).split(' '))>=3 else '')
        df['Activity time spent Zone 5'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[3] if len(str(x).split(' '))==4 else '')
    else:
        df['Activity time spent Zone 2'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=1 else '')
        df['Activity time spent Zone 3'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1] if len(str(x).split(' '))>=2 else '')
        df['Activity time spent Zone 4'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2] if len(str(x).split(' '))>=3 else '')

    if initia_col_len==10:
        df['Time spent O.H'] = df['Time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=1 else '')
        df['Time spent A.3rd'] = df['Time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=2 else '')
        df['Time spent P.A.'] = df['Time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2] if len(list(filter(lambda y: y.strip(), str(x).split(' '))))>=3 else '')
        
    df = df[['No', 'Name', 'Time', 'Distance covered (metres) Total', 'Distance covered (metres) In Poss', \
         'Distance covered (metres) Not in Poss','Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.', \
         'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent Zone 2', 'Activity time spent Zone 3',\
         'Activity time spent Zone 4', 'Activity time spent Zone 5']]

    dfs[c_file.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31]] = df
#     print(df)
    

In [ ]:
with pd.ExcelWriter('./russia_wc_teamstatistics_125.xlsx') as writer:
    for k in dfs.keys():
        dfs[k].to_excel(writer, \
                       sheet_name=k.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31],\
                       index=False)
#     df.to_excel(writer, sheet_name=c_file.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31], index=False)
#     df2.to_excel(writer, 'Data 1')

In [ ]:
# df.iloc[0]['Name'].find('\'')
# df.iloc[0]['Name'].find('\"')
df

In [ ]:
c_file = onlyfiles[1]
print(c_file)
player_statistic = textract.process(c_file)
lines = str(player_statistic).split('\\n')
country_id = lines[9].lower().strip()
if country_id not in country_names:
    country_id = lines[13].lower().strip()+'%'
#         print(country_id)

if lines[7].find('#')>=0:
    match_id = lines[7].split(' ')[1]
else:
    match_id = lines[11].split(' ')[1]
print('%s :::: %s or %s' % (country_id, lines[9], lines[13]))

df = tabula.read_pdf(c_file, multiple_tables=False)

df

In [ ]:
df.dtypes

In [ ]:
df = df[2:16]
df.columns = ['No', 'Name', 'Time', 'Distance covered (metres)', 'Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.',\
             'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent', 'Activity time spent Zone 5']

df

In [ ]:
df['Distance covered (metres) Total'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
df['Distance covered (metres) In Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1])
df['Distance covered (metres) Not in Poss'] = df['Distance covered (metres)'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2])

df['Activity time spent Zone 2'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[0])
df['Activity time spent Zone 3'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[1] if len(str(x).split(' '))>=2 else '')
df['Activity time spent Zone 4'] = df['Activity time spent'].apply(lambda x: list(filter(lambda y: y.strip(), str(x).split(' ')))[2] if len(str(x).split(' '))>=3 else '')

df

In [ ]:
df = df[['No', 'Name', 'Time', 'Distance covered (metres) Total', 'Distance covered (metres) In Poss', \
         'Distance covered (metres) Not in Poss','Time spent O.H', 'Time spent A.3rd', 'Time spent P.A.', \
         'SP', 'T.S.', 'Activity time spent Zone 1', 'Activity time spent Zone 2', 'Activity time spent Zone 3',\
         'Activity time spent Zone 4', 'Activity time spent Zone 5']]

df

In [ ]:
# # df.to_excel(writer, sheet_name=c_file.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31], index=False)
# df.to_excel('./russia_wc_teamstatistics.xlsx', sheet_name=c_file.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31], index=False)
# # writer.save()
dfs[c_file.replace('/Users/grace/workspace/football/russia_dataset//', '').replace('.pdf', '')[:31]] = df

## DB insertion

In [ ]:
for _dir in root_dirs:
    onlyfiles = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    onlyfiles = [_dir+'/'+x for x in onlyfiles if ('teamstatistics' in x and 'fwc' not in x)]

    for z in range(len(onlyfiles)):
        print('file z::: %s' % str(z))
        print(onlyfiles[z])
        player_statistic = textract.process(onlyfiles[z])
        lines = str(player_statistic).split('\\n')
        country_id = lines[9].lower().strip()
        if country_id not in country_names:
            country_id = lines[13].lower().strip()+'%'
#         print(country_id)

        if lines[7].find('#')>=0:
            match_id = lines[7].split(' ')[1]
        else:
            match_id = lines[11].split(' ')[1]
        print('%s :::: %s or %s' % (country_id, lines[9], lines[13]))

        df = tabula.read_pdf(onlyfiles[z], multiple_tables=False)  
    #     print(df.iloc[1].values.tolist())
    #     print(df.columns.__len__())

        col_criteria = df.iloc[1].values.tolist()
        merged_list = []
        display(df)
        print("-------")

        if col_criteria.__contains__('Total'):
            if col_criteria[2] == 'Total':
                merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1]].values.tolist(), columns = ['#', 'Name']))
                merged_list.append(pd.DataFrame(df[2:16]['Time Distance covered (metres)'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['Time', 'Total']))
                merged_list.append(pd.DataFrame(df[2:16]['Unnamed: 3'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['In Poss', 'Not in Poss']))
            elif col_criteria[3] == 'Total':
                merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1,2]].values.tolist(), columns = ['#', 'Name', 'Time']))
                merged_list.append(pd.DataFrame(df[2:16]['Unnamed: 4'].apply(lambda x: list(filter(None, x.split(' ')))).values.tolist(), columns = ['In Poss', 'Not in Poss']))

        else:
            merged_list.append(pd.DataFrame(df.iloc[2:16, [0,1,2]].values.tolist(), columns = ['#', 'Name', 'Time']))    
            merged_list.append(pd.DataFrame(df[2:16]['Distance covered (metres)'].apply(lambda x: list(filter(None, x.split(' ')))[:3]).values.tolist(), columns = ['Total', 'In Poss', 'Not in Poss']))


        if col_criteria.__contains__('O.H. A.3rd P.A.'):
            merged_list.append(pd.DataFrame(df[2:16]['Time spent'].apply(lambda x: list(filter(None, str(x).split(' ')))[:3]).values.tolist(), columns = ['O.H.', 'A.3rd', 'P.A.']))
        else:
            si = col_criteria.index('O.H.')
            ei = si + 3
            merged_list.append(pd.DataFrame(pd.DataFrame(df.iloc[2:16, si:ei].values.tolist(), columns = ['O.H.', 'A.3rd', 'P.A.'])))

        time_stat = pd.concat(merged_list, axis=1)
    #     display(time_stat)

        for index, row in time_stat.iterrows():
            num = row['#']
            print(num)
            if num != '':
                try:
#                     player_id = get_player_id(country_id, num.strip())['id']
                    player_id = country_id+':'+num.strip()
                    print(player_id)
                    if pd.notna(row['In Poss'].strip()) and row['In Poss']!='None':
        #                 print(row['In Poss'])
                        save_stat({'player_id':player_id,
                                    'match_id':match_id,
                                  'stat_type':'DCIP',
                                  'stat_value':row['In Poss'],
                                  'half':'total'})

                    if pd.notna(row['Not in Poss'].strip()) and row['Not in Poss']!='None':
        #                 print(row['Not in Poss'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'DCNIP',
                                  'stat_value':row['Not in Poss'],
                                  'half':'total'})
                    if pd.notna(row['O.H.']) and row['O.H.']!='None' and row['O.H.']!='nan':
        #                 print(row['O.H.'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TOH',
                                  'stat_value':row['O.H.'],
                                  'half':'total'})
                    if pd.notna(row['A.3rd']) and row['A.3rd']!='None' and row['A.3rd']!='nan':
        #                 print(row['A.3rd'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TA3RD',
                                  'stat_value':row['A.3rd'],
                                  'half':'total'})
                    if pd.notna(row['P.A.']) and row['P.A.']!='None' and row['P.A.']!='nan':
        #                 print(row['P.A.'])
                        save_stat({'player_id':player_id,
                                   'match_id':match_id,
                                  'stat_type':'TPA',
                                  'stat_value':row['P.A.'],
                                  'half':'total'})
                except Exception as e:
                    print(e)
                    print('================== player name missing =================')
                    print(row)                
            else:
                print('================== player name missing =================')
                print(row)